```
Copyright 2025 IBM Corporation

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# Credit Card Fraud Detection with Snap ML Boosting Machine Classifier and Triton-ibmsnapml

## Overview

Leveraging the Snap ML backend instead of the Python backend in Triton Inference Server to achieve optimized inference performance. Snap ML's C++ implementation ensures lower latency, better resource utilization, and efficient execution of ML models on IBM Z Systems. Unlike Python-based inference, Snap ML minimizes overhead, making it suitable for high-throughput and real-time applications.

This notebook is a step-by-step guide for data preprocessing, model implementation, and inference on the Credit Card Fraud Detection dataset. It demonstrates how to deploy a fraud detection model using Triton's ibmsnapml backend, covering the end-to-end workflow from data preparation to serving predictions.


## Pre-Requisites
Ensure you have the following installed and set up:

**Required Tools:**
 - Docker
 - IBM Cloud account
 - Python 3.10

**Python Dependencies:**
  - jupyter
  - plotly
  - numpy
  - pandas
  - pip
  - scikit-learn
  - snapml
  - requests

### Fetching Credit Card Transaction Dataset
1. The details of the base dataset can be found in the [Credit_Card_Fraud_Detection](https://www.kaggle.com/datasets/vineetdumir/credit-card-fraud-detection/data).
2. Generated Synthetic data to balance the underrepresented minority classes.

**NOTE**: Please make sure to download the synthetic credit card transaction dataset provided in [credit_card](https://ibm.box.com/v/tabformer-data) to proceed with further steps, and the dataset contains 24M records across 12 fields.

In [1]:
import os
import zipfile
import tarfile

# Extract the zip file
zip_file_name = "credit_card.zip"  # Replace with the actual .zip file name
current_dir = os.getcwd()

with zipfile.ZipFile(zip_file_name, "r") as zip_ref:
    zip_folder_path = os.path.join(current_dir, os.path.splitext(zip_file_name)[0])
    zip_ref.extractall(current_dir)

# Locate and extract the .tgz file inside the extracted folder
for file in os.listdir(zip_folder_path):
    if file.endswith(".tgz"):
        tgz_file_path = os.path.join(zip_folder_path, file)
        with tarfile.open(tgz_file_path, "r:gz") as tgz_ref:
            tgz_ref.extractall(zip_folder_path)
            print(f"Extracted {tgz_file_path} to {zip_folder_path}")
        break
else:
    print("No .tgz file found in the extracted zip folder.")

# Locate the .csv file
for file in os.listdir(zip_folder_path):
    if file.endswith(".csv"):
        csv_file_path = os.path.join(zip_folder_path, file)
        print(f"Found CSV file: {csv_file_path}")
        break
else:
    print("No CSV file found.")

/usr/lib64/python3.9/tarfile.py:2268: RuntimeWarning: The default behavior of tarfile extraction has been changed to disallow common exploits (including CVE-2007-4559). By default, absolute/parent paths are disallowed and some mode bits are cleared. See https://access.redhat.com/articles/7004769 for more details.
  warnings.warn(


Extracted /home/hemaprasannakc/kanaka/demo/CCF-e2e-pipeline_snapml_BM/credit_card/transactions.tgz to /home/hemaprasannakc/kanaka/demo/CCF-e2e-pipeline_snapml_BM/credit_card
Found CSV file: /home/hemaprasannakc/kanaka/demo/CCF-e2e-pipeline_snapml_BM/credit_card/card_transaction.v1.csv


### Transform Synthetic Data for Model Preparation:

Apply the following transformations to prepare the dataset for modeling:

1. Extract 'Hour', and 'Minute' from the Time column.
2. Remove the '$' symbol from the Amount column.
3. Replace NaN values in the Merchant State, Errors?, and Zip columns with NoState, NoError, and NoZip, respectively.
4. Remove spaces from the labels in columns such as Use Chip, Merchant City, Merchant State, and Errors?.
5. Encode the Merchant Name column using encoded integer values.

In [2]:
# Import the pandas library for data manipulation
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# Load the data from a CSV file into a DataFrame
data = pd.read_csv(csv_file_path)

In [3]:
# Display the first few rows of the DataFrame to inspect the data structure
data.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No


In [4]:
# Display the shape of the DataFrame (number of rows and columns)
data.shape

(24386900, 15)

In [5]:
# Display basic information about the DataFrame, including data types and memory usage
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24386900 entries, 0 to 24386899
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   User            int64  
 1   Card            int64  
 2   Year            int64  
 3   Month           int64  
 4   Day             int64  
 5   Time            object 
 6   Amount          object 
 7   Use Chip        object 
 8   Merchant Name   int64  
 9   Merchant City   object 
 10  Merchant State  object 
 11  Zip             float64
 12  MCC             int64  
 13  Errors?         object 
 14  Is Fraud?       object 
dtypes: float64(1), int64(7), object(7)
memory usage: 2.7+ GB


In [6]:
# Display column names of the DataFrame
data.columns

Index(['User', 'Card', 'Year', 'Month', 'Day', 'Time', 'Amount', 'Use Chip',
       'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC',
       'Errors?', 'Is Fraud?'],
      dtype='object')

In [7]:
data['Is Fraud?'].value_counts()

Is Fraud?
No     24357143
Yes       29757
Name: count, dtype: int64

In [8]:
#Raw Data Transformation:
def raw_data_transformation(df, file_path=None):
    """
    Preprocesses the given DataFrame by applying transformations and saves it to a specified file path.

    Parameters:
    - df (pd.DataFrame): Input DataFrame to preprocess.
    - file_path (str): Path to save the processed DataFrame.

    Returns:
    - pd.DataFrame: The processed DataFrame.
    """
    # Check if the column `time` exists and validate its format
    if 'Time' in df.columns:
        # Split 'Time' into 'Hour' and 'Minute' and 'Second' if exists
        if df['Time'].str.match(r'^\d{1,2}:\d{1,2}$').all():  # h:m format
            df[['Hour', 'Minute']] = df['Time'].str.split(':', expand=True)
            df = df.drop(columns=['Time'])  # Drop the original Time column
            df[['Hour', 'Minute']] = df[['Hour', 'Minute']].astype(int)  # Convert to integers
        elif df['Time'].str.match(r'^\d{1,2}:\d{1,2}:\d{1,2}$').all():  # h:m:s format
            df[['Hour', 'Minute', 'Second']] = df['Time'].str.split(':', expand=True)
            df = df.drop(columns=['Time'])  # Drop the original Time column
            df[['Hour', 'Minute', 'Second']] = df[['Hour', 'Minute', 'Second']].astype(int)  # Convert to integers
        else:
            print("The Time column contains invalid format.")
    
    # Remove the '$' from the amount column and convert to numeric
    if df['Amount'].dtype == 'object' and df['Amount'].str.contains(r"\$", regex=True).any():
        df['Amount'] = (
            df['Amount']
            .astype(str)                   # Ensure all values are strings
            .str.replace(r"\$", "", regex=True)  # Remove the dollar sign
            .astype(float)                 # Convert to float
        )
    
    # Replace specific values in 'Transaction Type'
    df["Use Chip"] = df["Use Chip"].replace({
        "Swipe Transaction": "SwipeTransaction",
        "Chip Transaction": "ChipTransaction",
        "Online Transaction": "OnlineTransaction"
    })

    # Remove non-alphanumeric characters from specific columns
    df["Merchant City"] = df["Merchant City"].str.replace(r"[^a-zA-Z0-9]+", "", regex=True)
    df["Merchant State"] = df["Merchant State"].str.replace(r"[^a-zA-Z0-9]+", "", regex=True)
    df["Errors?"] = df["Errors?"].str.replace(r"[^a-zA-Z0-9]+", "", regex=True)

    # Fill missing values with respective labels
    df["Merchant State"] = df["Merchant State"].fillna("NoState")
    df["Errors?"] = df["Errors?"].fillna("NoError")
    df["Zip"] = df["Zip"].fillna(0)

    #Replaces 'No' with 0 and 'Yes' with 1 in the Is Fraud? column.
    if 'Is Fraud?' in df.columns: df['Is Fraud?'] = df['Is Fraud?'].replace({"No": 0, "Yes": 1})

    # Specify the desired column order
    new_column_order = ['User', 'Card', 'Year', 'Month', 'Day', 'Hour', 'Minute',
                        'Amount', 'Use Chip', 'Merchant Name', 'Merchant City', 
                        'Merchant State', 'Zip', 'MCC', 'Errors?', 'Is Fraud?']
    
    # Rearrange the DataFrame columns
    df = df[new_column_order]

    # Save the processed DataFrame to the specified file path
    if file_path: df.to_csv(file_path, index=False)
    return df

In [9]:
%%time
tdf_trans = raw_data_transformation(data)
tdf_trans.head()

CPU times: user 1min 53s, sys: 3.22 s, total: 1min 57s
Wall time: 1min 57s


,User,Card,Year,Month,Day,Hour,Minute,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,6,21,134.09,SwipeTransaction,3527213246127876953,LaVerne,CA,91750.0,5300,NoError,0
1,0,0,2002,9,1,6,42,38.48,SwipeTransaction,-727612092139916043,MontereyPark,CA,91754.0,5411,NoError,0
2,0,0,2002,9,2,6,22,120.34,SwipeTransaction,-727612092139916043,MontereyPark,CA,91754.0,5411,NoError,0
3,0,0,2002,9,2,17,45,128.95,SwipeTransaction,3414527459579106770,MontereyPark,CA,91754.0,5651,NoError,0
4,0,0,2002,9,3,6,23,104.71,SwipeTransaction,5817218446178736267,LaVerne,CA,91750.0,5912,NoError,0


In [10]:
tdf_trans.shape

(24386900, 16)

In [11]:
tdf_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24386900 entries, 0 to 24386899
Data columns (total 16 columns):
 #   Column          Dtype  
---  ------          -----  
 0   User            int64  
 1   Card            int64  
 2   Year            int64  
 3   Month           int64  
 4   Day             int64  
 5   Hour            int64  
 6   Minute          int64  
 7   Amount          float64
 8   Use Chip        object 
 9   Merchant Name   int64  
 10  Merchant City   object 
 11  Merchant State  object 
 12  Zip             float64
 13  MCC             int64  
 14  Errors?         object 
 15  Is Fraud?       int64  
dtypes: float64(2), int64(10), object(4)
memory usage: 2.9+ GB


In [12]:
tdf_trans.isna().sum()

User              0
Card              0
Year              0
Month             0
Day               0
Hour              0
Minute            0
Amount            0
Use Chip          0
Merchant Name     0
Merchant City     0
Merchant State    0
Zip               0
MCC               0
Errors?           0
Is Fraud?         0
dtype: int64

### Data Processing and Model Preparation Steps:

1. **Preprocessing Pipeline Implementation**: For data preprocessing, applying various transformations such as normalization, and categorical encoders. The goal is to ensure that the data is in a suitable format and quality for training the model.
2. **Model Pipeline Implementation**: For model preparation, configuring its parameters, and preparing it for training on the preprocessed data.
3. **Model Evaluation**: Perform predictions and post-process results for evaluation.

In [13]:
# Separating features (X_tdf) and target variable (y_tdf)
X_tdf = tdf_trans.drop("Is Fraud?", axis=1)
y_tdf = tdf_trans["Is Fraud?"]

In [14]:
# Converting selected columns to string type
X_tdf['Merchant Name'] = X_tdf['Merchant Name'].astype(str)
X_tdf['Merchant State'] = X_tdf['Merchant State'].astype(str)
X_tdf['Zip'] = X_tdf['Zip'].astype(pd.Int64Dtype()).astype(str)
X_tdf['MCC'] = X_tdf['MCC'].astype(str)
X_tdf['Errors?'] = X_tdf['Errors?'].astype(str)

In [15]:
# Displaying information about the DataFrame including data types and memory usage
X_tdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24386900 entries, 0 to 24386899
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   User            int64  
 1   Card            int64  
 2   Year            int64  
 3   Month           int64  
 4   Day             int64  
 5   Hour            int64  
 6   Minute          int64  
 7   Amount          float64
 8   Use Chip        object 
 9   Merchant Name   object 
 10  Merchant City   object 
 11  Merchant State  object 
 12  Zip             object 
 13  MCC             object 
 14  Errors?         object 
dtypes: float64(1), int64(7), object(7)
memory usage: 2.7+ GB


#### Fraud vs Non-Fraud Distribution in the Train Dataset

In [16]:
import plotly.graph_objects as go
import numpy as np

# Assuming `y_tdf` contains the fraud labels (0 for Non-Fraud, 1 for Fraud)
labels = ['Non-Fraud', 'Fraud']
sizes = [np.sum(y_tdf == 0), np.sum(y_tdf == 1)]
colors = ['#8fd694', '#ff6f61']  # Custom colors

# Create the pie chart
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=sizes, 
    marker=dict(colors=colors), 
    pull=[0, 0.05] if sizes[1] > 0 else [0, 0],  # Slightly separate Fraud
    hoverinfo='label+percent+value',  # Show label, percentage, and count on hover
    textinfo='percent+value'  # Show percentage and count on the chart
)])

# Customize the layout
fig.update_layout(
    title="Fraud vs Non-Fraud Distribution in the Dataset",
    title_font_size=16
)

# Show the plot
fig.show()

#### 1. **Preprocessing Pipeline Implementation**:

In [17]:
# Defining a pipeline consisting of data preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer, OrdinalEncoder, OneHotEncoder, KBinsDiscretizer, TargetEncoder
preprocess_pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            # Normalize user-related numerical features
            ('numeric_normalizer', Normalizer(), ['User', 'Card', 'Amount']),
            
            # Discretize numeric and time features
            ('daytime_encoder', KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile'),
             ['Year', 'Month', 'Day', 'Hour', 'Minute']),
            
            # One-hot encode 'Use Chip' (has only 3 unique values)
            ('transaction_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['Use Chip']),

            # Target encoding for other features like 'Merchant' 
            ('merchant_tarencoder', TargetEncoder(), ['Merchant Name', 'Merchant City', 'Merchant State']),
            
            # Ordinal encoding for categorical features
            ('merchant_ordencoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ['Zip', 'MCC', 'Errors?']),
        ]
    )),
])

In [18]:
# Preprocess pipeline training
preprocess_pipeline.fit(X_tdf, y_tdf)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric_normalizer',
                                                  Normalizer(),
                                                  ['User', 'Card', 'Amount']),
                                                 ('daytime_encoder',
                                                  KBinsDiscretizer(encode='ordinal'),
                                                  ['Year', 'Month', 'Day',
                                                   'Hour', 'Minute']),
                                                 ('transaction_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Use Chip']),
                                                 ('merchant_tarencoder',
                                                  TargetEncoder(),
                                                  ['Merchant Name',
                                                   'Merchant City',
                                                   'Merchant State']),
                                                 ('merchant_ordencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['Zip', 'MCC',
                                                   'Errors?'])]))])

In [19]:
# Apply the preprocessing on the training data
X_train_preprocessed = preprocess_pipeline.transform(X_tdf)

#### 2. **Model Pipeline Implementation**:

In [20]:
%%time
from snapml import BoostingMachineClassifier
from sklearn.model_selection import train_test_split

# Split the data
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train_preprocessed, y_tdf, test_size=0.2, stratify=y_tdf, random_state=42
)

# Initialize the SnapML model with your parameters
model = BoostingMachineClassifier(
    n_jobs=4,
    num_round=700,
    max_depth=5,
    learning_rate=0.05,
    gamma=1.0,
    lambda_l2=15.0,
    subsample=0.8,
    colsample_bytree=0.8,
    early_stopping_rounds=50,
    random_state=42,
    verbose=True,  # Enable training progress logs
    class_weight='balanced'
)

# Train the SnapML model with a validation set
model.fit(
    X=X_train_split, 
    y=y_train_split,
    X_val=X_val,          # Validation features
    y_val=y_val,          # Validation labels
    aggregate_importances=True  # For feature importance calculation
)

it:      0, learner: t, loss_train: 6.587320e-01, loss_val: 6.588358e-01, it_since_best:      0 
it:      1, learner: t, loss_train: 6.220541e-01, loss_val: 6.223904e-01, it_since_best:      0 
it:      2, learner: t, loss_train: 5.868370e-01, loss_val: 5.873238e-01, it_since_best:      0 
it:      3, learner: t, loss_train: 5.574592e-01, loss_val: 5.579906e-01, it_since_best:      0 
it:      4, learner: t, loss_train: 5.278555e-01, loss_val: 5.285120e-01, it_since_best:      0 
it:      5, learner: t, loss_train: 5.063648e-01, loss_val: 5.071309e-01, it_since_best:      0 
it:      6, learner: t, loss_train: 4.807671e-01, loss_val: 4.817163e-01, it_since_best:      0 
it:      7, learner: t, loss_train: 4.573084e-01, loss_val: 4.584562e-01, it_since_best:      0 
it:      8, learner: t, loss_train: 4.354895e-01, loss_val: 4.367727e-01, it_since_best:      0 
it:      9, learner: t, loss_train: 4.172691e-01, loss_val: 4.185873e-01, it_since_best:      0 
it:     10, learner: t, loss_t

BoostingMachineClassifier(class_weight='balanced', colsample_bytree=0.8,
                          early_stopping_rounds=50, lambda_l2=15.0,
                          learning_rate=0.05, max_depth=5, n_jobs=4,
                          num_round=700, random_state=42, subsample=0.8,
                          verbose=True)

s_val: 1.405583e-01, it_since_best:      0 
it:     52, learner: t, loss_train: 1.341777e-01, loss_val: 1.382953e-01, it_since_best:      0 
it:     53, learner: t, loss_train: 1.323509e-01, loss_val: 1.365611e-01, it_since_best:      0 
it:     54, learner: t, loss_train: 1.307770e-01, loss_val: 1.349928e-01, it_since_best:      0 
it:     55, learner: t, loss_train: 1.294058e-01, loss_val: 1.336455e-01, it_since_best:      0 
it:     56, learner: t, loss_train: 1.278221e-01, loss_val: 1.321220e-01, it_since_best:      0 
it:     57, learner: t, loss_train: 1.259996e-01, loss_val: 1.303369e-01, it_since_best:      0 
it:     58, learner: t, loss_train: 1.247160e-01, loss_val: 1.290537e-01, it_since_best:      0 
it:     59, learner: t, loss_train: 1.233444e-01, loss_val: 1.277182e-01, it_since_best:      0 
it:     60, learner: t, loss_train: 1.222504e-01, loss_val: 1.266494e-01, it_since_best:      0 
it:     61, learner: t, loss_train: 1.208431e-01, loss_val: 1.252436e-01, it_since_

155704e-02, it_since_best:      0 
it:    155, learner: t, loss_train: 8.467744e-02, loss_val: 9.118697e-02, it_since_best:      0 
it:    156, learner: t, loss_train: 8.453565e-02, loss_val: 9.108838e-02, it_since_best:      0 
it:    157, learner: t, loss_train: 8.444486e-02, loss_val: 9.102721e-02, it_since_best:      0 
it:    158, learner: t, loss_train: 8.430096e-02, loss_val: 9.091207e-02, it_since_best:      0 
it:    159, learner: t, loss_train: 8.410155e-02, loss_val: 9.073129e-02, it_since_best:      0 
it:    160, learner: t, loss_train: 8.404763e-02, loss_val: 9.069153e-02, it_since_best:      0 
it:    161, learner: t, loss_train: 8.398623e-02, loss_val: 9.066026e-02, it_since_best:      0 
it:    162, learner: t, loss_train: 8.387805e-02, loss_val: 9.060231e-02, it_since_best:      0 
it:    163, learner: t, loss_train: 8.382391e-02, loss_val: 9.057199e-02, it_since_best:      0 
it:    164, learner: t, loss_train: 8.362950e-02, loss_val: 9.037334e-02, it_since_best:    

#### 3. Model Evaluation:

In [21]:
%%time
# Import necessary libraries for evaluation
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Make predictions on the preprocessed training data
y_pred = model.predict(X_train_preprocessed)

CPU times: user 11min 13s, sys: 457 ms, total: 11min 14s
Wall time: 2min 49s


In [22]:
%%time
# Import necessary libraries for evaluation
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Make predictions on the preprocessed training data
y_pred = model.predict(X_train_preprocessed)

CPU times: user 11min 15s, sys: 455 ms, total: 11min 15s
Wall time: 2min 49s


In [23]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
cm = confusion_matrix(y_tdf, y_pred)
labels = ['Non-Fraud', 'Fraud']

# Create a DataFrame for a tabular view
cm_df = pd.DataFrame(cm, index=labels, columns=labels)

# Print the confusion matrix as a table
print("Confusion Matrix:")
print(cm_df)

Confusion Matrix:
           Non-Fraud   Fraud
Non-Fraud   23798897  558246
Fraud            504   29253


In [24]:
# Generate classification report
from IPython.display import display

report_dict = classification_report(y_tdf, y_pred, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
display(report_df)

,precision,recall,f1-score,support
0,0.999979,0.977081,0.988397,2.435714e+07
1,0.049792,0.983063,0.094784,2.975700e+04
accuracy,0.977088,0.977088,0.977088,9.770881e-01
macro avg,0.524886,0.980072,0.541591,2.438690e+07
weighted avg,0.998819,0.977088,0.987307,2.438690e+07


In [25]:
# Free up some space by explicitly deleting dataframes
import gc
del data
del tdf_trans
del y_tdf
del X_train_split
del y_train_split
del X_val
del y_val
del X_train_preprocessed
gc.collect()

37

### Deploying Models on Triton Inference Server
Deploy the model and preprocessing pipeline for real-time inferencing using Triton's **ibmsnapml** backend.

#### Model Serialization
Save models in the directory structure required by Triton.
 - Models can be stored locally or in S3, Google Cloud Storage, or Azure Storage. For this example, we will stick to local storage.
 - Use numbered directories for multiple model versions.

In [26]:
import os
# Create the model repository directory. The name of this directory is arbitrary.
REPO_PATH = os.path.abspath('model_repository')
os.makedirs(REPO_PATH, exist_ok=True)

In [27]:
def serialize_model(model, model_name, preprocessor=None):
    # The name of the model directory determines the name of the model as reported
    # by Triton
    model_dir = os.path.join(REPO_PATH, model_name)
    # We can store multiple versions of the model in the same directory. In our
    # case, we have just one version, so we will add a single directory, named '1'.
    version_dir = os.path.join(model_dir, '1')
    os.makedirs(version_dir, exist_ok=True)

    if preprocessor:
        # The default filename for the preprocessor saved in json format is 'pipeline.json'.
        # Preprocess pipeline Saving
        from snapml import export_preprocessing_pipeline
        preprocess_file = os.path.join(version_dir, 'pipeline.json')
        # export preprocessing pipeline to JSON format
        export_preprocessing_pipeline(preprocessor, preprocess_file)
    # The default filename for the BoostingMachineClassifier model saved in pmml format is 'model.pmml'.
    # It is recommended that you use this filename to avoid having to specify a
    # name in the configuration file.
    model_file = os.path.join(version_dir, 'model.pmml')
    model.export_model(model_file)
    
    return model_dir

In [28]:
model_name = 'ccf_bmc_model'
model_dir = serialize_model(model, model_name, preprocess_pipeline['preprocessor'])

In [29]:
# View pipeline JSON data to validate the pre-process pipeline transformations 
# import json

# preprocess_pipeline_filepath = os.path.join(REPO_PATH, model_name, '1', 'pipeline.json')
# with open(preprocess_pipeline_filepath) as f:
#     pipeline_config = json.load(f)
    
# print(json.dumps(pipeline_config, indent=2))

#### The Configuration File
The configuration file for a model provides Triton with key details about the model and specifies how it should be used. While you can explore all general Triton configuration options [here](https://github.com/triton-inference-server/server/blob/master/docs/model_configuration.md). The configuration options specific to the **ibmsnapml** backend [here](https://github.com/IBM/ibmz-accelerated-for-nvidia-triton-inference-server?tab=readme-ov-file#model-configuration), this example focuses on a few of the most common and essential options.

Below are general descriptions of these options:
- **backend**: This parameter must be provided as “ibmsnapml” while utilising Snap ML C++ Backend.
- **max_batch_size**: The maximum batch size that can be passed to this model. In general, the only limit on the size of batches passed to an ibmsnapml backend is the memory available with which to process them.
- **input**: Options in this section tell Triton the number of features to expect for each input sample.
- **output**: Options in this section tell Triton how many output values there will be for each sample. If the "PREDICT_PROBABILITY" option (described further on) is set to true, then a probability value will be returned for each class. Otherwise, a single value will be returned indicating the class predicted for the given sample.
- **instance_group**: This determines how many instances of this model will be created.
- **PREPROCESSING_FILE**: This configuration file parameter specifies the name of the file to be used for preprocessing. The preprocessing file must be named as ‘pipeline.json’ when preprocessing is selected for end-to-end inferencing.
- **SNAPML_TARGET_CLASS**: This configuration parameter specifies which model needs to be imported by Snap ML C++ Backend as per Snap ML documentation [here](https://snapml.readthedocs.io/en/latest/model_import.html).
- **MODEL_FILE_FORMAT**: A string indicating what format the model is in ("xgb_json" in this example). Refer to Snap ML documentation [here](https://snapml.readthedocs.io/en/latest/model_import.html).
- **PREDICT_PROBABILITY**: If set to true, probability values will be returned for each class rather than just a class prediction.
- **NUM_OF_PREDICT_THREADS**: This defines the CPU threads running for each inference call.

Based on this information, let's set up configuration files for our models.

In [30]:
def generate_config(features, num_classes, model_dir, deployment_type='cpu', storage_type='AUTO'):
    if deployment_type.lower() == 'cpu':
        instance_kind = 'KIND_CPU'
    else:
        return "Supproting CPU only"

    config_text = f"""backend: "ibmsnapml"
max_batch_size: {max_batch_size}
input [                                 
 {{  
    name: "IN0"
    data_type: TYPE_STRING
    dims: [ {features} ]                    
  }} 
]
output [
 {{
    name: "OUT0"
    data_type: TYPE_FP64
    dims: [ {num_classes} ]
  }}
]
instance_group [
  {{ 
    count: 1
    kind: {instance_kind} 
  }}
]
parameters [
  {{
    key: "PREPROCESSING_FILE"
    value: {{ string_value: "pipeline.json" }}
  }},
  {{
    key: "SNAPML_TARGET_CLASS"
    value: {{ string_value: "BoostingMachineClassifier" }}
  }},
  {{
    key: "MODEL_FILE_FORMAT"
    value: {{ string_value: "pmml" }}
  }},
  {{
    key: "PREDICT_PROBABILITY"
    value: {{ string_value: "false" }}
  }},
  {{
    key: "NUM_OF_PREDICT_THREADS"
    value: {{ string_value: "4" }}
  }}
]

dynamic_batching {{
  max_queue_delay_microseconds: 25000
}}"""
    config_path = os.path.join(model_dir, 'config.pbtxt')
    with open(config_path, 'w') as file_:
        file_.write(config_text)

    return config_path

In [31]:
features = X_tdf.shape[1]
max_batch_size = 32 
num_classes = 1 #As we have two classes and setting PREDICT_PROBABILITY as false
generate_config(features, num_classes, model_dir, deployment_type='cpu')

'/home/hemaprasannakc/kanaka/demo/CCF-e2e-pipeline_snapml_BM/model_repository/ccf_bmc_model/config.pbtxt'

In [32]:
# View Model Repository Structure
from pathlib import Path

def print_tree(path):
    print(f"{path}/")
    for p in sorted(Path(path).rglob('*')):
        depth = len(p.relative_to(path).parts)
        spacer = '    ' * depth
        print(f"{spacer}{p.name}")

print_tree('model_repository')

model_repository/
    ccf_bmc_model
        1
            model.pmml
            pipeline.json
        config.pbtxt


In [33]:
# View Model configuration
from IPython.display import Code

with open('model_repository/ccf_bmc_model/config.pbtxt', 'r') as f:
    display(Code(f.read(), language='protobuf'))

backend: "ibmsnapml"
max_batch_size: 32
input [                                 
 {  
    name: "IN0"
    data_type: TYPE_STRING
    dims: [ 15 ]                    
  } 
]
output [
 {
    name: "OUT0"
    data_type: TYPE_FP64
    dims: [ 1 ]
  }
]
instance_group [
  { 
    count: 1
    kind: KIND_CPU 
  }
]
parameters [
  {
    key: "PREPROCESSING_FILE"
    value: { string_value: "pipeline.json" }
  },
  {
    key: "SNAPML_TARGET_CLASS"
    value: { string_value: "BoostingMachineClassifier" }
  },
  {
    key: "MODEL_FILE_FORMAT"
    value: { string_value: "pmml" }
  },
  {
    key: "PREDICT_PROBABILITY"
    value: { string_value: "false" }
  },
  {
    key: "NUM_OF_PREDICT_THREADS"
    value: { string_value: "4" }
  }
]

dynamic_batching {
  max_queue_delay_microseconds: 25000
}

#### Download the IBM Z Accelerated for NVIDIA Triton™ Inference Server Container image
 - Log in to the IBM Container Registry ([icr.io](https://ibm.github.io/ibm-z-oss-hub/containers/ibmz-accelerated-for-nvidia-triton-inference-server.html)) using valid credentials.
 - Pull the [IBM Z Accelerated for NVIDIA Triton™ Inference Server Container image](https://ibm.github.io/ibm-z-oss-hub/containers/ibmz-accelerated-for-nvidia-triton-inference-server.html).
 - Refer to the IBM documentation for details on obtaining credentials [here](https://ibm.github.io/ibm-z-oss-hub/main/main.html).

In [ ]:
# Replace with your keyname and API key
keyname = "iamapikey"
apikey = "<your-apikey>"
registry_url = "http://icr.io/"

# Perform Docker login using the API key
!echo {apikey} | docker login -u {keyname} {registry_url} --password-stdin

# Pull a Docker image from the registry
TRITON_IMAGE = "icr.io/ibmz/ibmz-accelerated-for-nvidia-triton-inference-server:1.4.0" #Latest version of IBM Z Accelerated for NVIDIA Triton™ Inference Server container image
!docker pull {TRITON_IMAGE}

Login Succeeded


1.4.0: Pulling from ibmz/ibmz-accelerated-for-nvidia-triton-inference-server


2c7366af510b: Pulling fs layer 


0b80c99c521f: Pulling fs layer 


82f34b8a024a: Pulling fs layer 


a2ae4227ce50: Pulling fs layer 


1f4f117092d1: Pulling fs layer 


3b0ff6cadca3: Pulling fs layer 


6ce88ceb0a45: Pulling fs layer 


f46d6ba18537: Pulling fs layer 


3486716f0da8: Pulling fs layer 

3b0ff6cadca3: Waiting 

1f4f117092d1: Waiting 

f46d6ba18537: Waiting 

6ce88ceb0a45: Waiting 


5d96a0541b22: Pulling fs layer 

3486716f0da8: Waiting 


5788b4996ab2: Pulling fs layer 

5d96a0541b22: Waiting 

a2ae4227ce50: Waiting 


1f3479c096cd: Pulling fs layer 


3f786e57c609: Pulling fs layer 

5788b4996ab2: Waiting 

1f3479c096cd: Waiting 


2c19fac2c1a4: Pulling fs layer 


923204231833: Pulling fs layer 


b0ab249dfe39: Pulling fs layer 


6edc9f84dc24: Pulling fs layer 

3f786e57c609: Waiting 


5e0ea807c814: Pulling fs layer 

2c19fac2c1a4: Waiting 


9e3023d788aa: Pulling fs layer 

923204231833


0b80c99c521f: Downloading  147.5kB/14.27MB

82f34b8a024a: Downloading    213kB/19.83MB

2c7366af510b: Downloading  286.7kB/28.65MB



0b80c99c521f: Downloading  3.391MB/14.27MB

82f34b8a024a: Downloading  3.342MB/19.83MB



2c7366af510b: Downloading  2.351MB/28.65MB


...


16c449db3d20: 

Extracting  5.225MB/5.225MB

16c449db3d20: Pull complete 

b2935665fa91: Extracting     239B/239B

b2935665fa91: Extracting     239B/239B

b2935665fa91: Pull complete 

53e56fcc278f: Extracting     290B/290B

53e56fcc278f: Extracting     290B/290B

53e56fcc278f: Pull complete 

3cd6e26ec78c: Extracting  32.77kB/349kB

3cd6e26ec78c: Extracting    349kB/349kB

3cd6e26ec78c: Extracting    349kB/349kB

3cd6e26ec78c: Pull complete 

7915f3cfcd7e: Extracting     425B/425B

7915f3cfcd7e: Extracting     425B/425B

7915f3cfcd7e: Pull complete 

abf503969137: Extracting  229.4kB/19.9MB



abf503969137: Extracting  8.946MB/19.9MB



abf503969137: Extracting  17.43MB/19.9MB



abf503969137: Extracting   19.9MB/19.9MB

abf503969137: Pull complete 
Digest: sha256:484c5f9aeff0c911d110ae85596aa33a5ac3b594f561b712d9cf7440b8a6d8b3
Status: Downloaded newer image for icr.io/ibmz/ibmz-accelerated-for-nvidia-triton-inference-server:1.4.0
icr.io/ibmz/ibmz-accelerated-for-nvidia-triton-inference-server:1.4.0


#### Starting the Triton Inference Server
 - Launch the Triton server with the model and configuration files in place.
 - Use Python to verify server readiness and check logs for warnings or errors.

In [35]:
!docker run --shm-size 1G -itd -p 8000:8000 -p 8001:8001 -p 8002:8002 -v {REPO_PATH}:/models --name tritonserver {TRITON_IMAGE} tritonserver --model-repository=/models

cc033bd72bf1b4c6f393a5d9c01536ada5ab0f7812b3a3c28c2c28e359e5efc3


In [36]:
import time
import requests

def check_server_status(server_url, num_retries=30, sleep_time=1):
    """
    Use Triton's ready endpoint to verify that the server and the models are ready for inference.
    Using curl to access the HTTP endpoint that indicates server status.
    Parameters:
        num_retries (int): number of times to send a ready status
                                request to the server before raising
                                an exception
        sleep_time (int): amount of time in seconds to sleep between retries
    Returns:
    bool: False if the server is not ready even after num_retries retries, otherwise True.   
    """

    retries = num_retries
    server_status_api = f"http://{server_url}/v2/health/ready"

    while retries > 0:
        try:
            response = requests.get(server_status_api)
            time.sleep(sleep_time)
            if response.status_code == 200:
                return True
            else:
                retries -= 1
        except requests.RequestException as e:
            time.sleep(sleep_time)
            retries -= 1
            print(f"Server not yet ready, encountered error: {e}")
    return False

In [37]:
#Server Health ready check
HOST = 'localhost'
PORT = 8000

server_status = check_server_status(server_url=f'{HOST}:{PORT}')
if not server_status:
    raise RuntimeError(("Failed to start Triton server. Check the log for details."))
else:
    print("Triton Inference Server started successfully.")

Server not yet ready, encountered error: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


Triton Inference Server started successfully.


In [38]:
!docker logs tritonserver

=-=-=-=-=-= IBM Z Accelerated for NVIDIA Triton Inference Server =-=-=-=-=-=-=-=-=-=-=-=-=-=

Version: 1.4.0

License: https://github.com/IBM/ibmz-accelerated-for-nvidia-triton-inference-server#Licenses

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-



I0630 11:04:07.514642 1 model_lifecycle.cc:472] "loading: ccf_bmc_model:1"

I0630 11:04:07.518120 1 snapml_model_state.cc:241] "Loading /models/ccf_bmc_model/1/model.pmml"

I0630 11:04:07.518134 1 snapml_model_state.cc:256] "Loading /models/ccf_bmc_model/1/pipeline.json"

I0630 11:04:07.518201 1 snapml_model_instance_state.cc:110] "TRITONBACKEND_ModelInstanceInitialize: ccf_bmc_model_0_0 (CPU device 0)"

I0630 11:04:07.698086 1 model_lifecycle.cc:838] "successfully loaded 'ccf_bmc_model'"

I0630 11:04:07.698128 1 server.cc:604] 

+------------------+------+

| Repository Agent | Path |

+------------------+------+

+------------------+------+



I0630 11:04:07.698157 1 server.cc:631] 

+----

#### Submitting inference requests
 - Send inference requests to the Triton server.
 - Validate results by comparing them with local predictions.
 - Slight deviations might occur due to floating-point computations in parallel execution while choosing to Predict probability.

In [39]:
def get_response(server_url, sample_data, model_name, model_version=None):
    
    headers = {
        "Content-Type": "application/json"
    }

    model_version = model_version if model_version is not None else 1
    model_inference_url = f"http://{server_url}/v2/models/{model_name}/versions/{model_version}/infer"

    response = {
        "response_obj" : None,
        "status_code" : None, 
        "error_reason": None
    }
    resp = None

    payload = {
        "inputs": [
            {
                "name": "IN0",
                "shape": [2, 15],
                "datatype": "BYTES",
                "data": sample_data
            }
        ],
        "outputs": [
            {
                "name": "OUT0",
                "shape": [2,1],
                "datatype": "FP64"
            }
        ]
    }

    headers = {
        "Content-Type": "application/json"
    }

    print(f"\nMaking inference call to {model_inference_url} with data \n {payload} \n\n ")

    try: 
        resp = requests.post(
                        model_inference_url,
                        headers=headers,
                        json=payload
                    )
        if resp.status_code != 200:
            raise Exception(f"HTTP request failed with {resp.status_code}")
        
        response["response_obj"] = resp.json()
        response["status_code"] = resp.status_code
        response["error_reason"] = None

    except Exception as e:
        # still fecth the error info and create a responce object 
        if resp is not None:
            response["response_obj"] = resp.json() 
            response["status_code"] = resp.status_code
        else:
            response["status_code"] = 400 # Client side Exception
        response["error_reason"] = e

        return response
    return response

In [40]:
#We have considered sample data each sample representing the Non-Fraud(0) and Fraud(1) classes respectively.
model_name = 'ccf_bmc_model'
sample_data = [["525", "4388509355512107", "2005", "11", "8", "7", "29", "43.42", "OnlineTransaction", "31551052261259716", "ONLINE", "NoState", "0", "4784", "NoError"],
               ["1673", "5927292790575653", "2005", "12", "5", "10", "44", "35.75", "SwipeTransaction", "1913477460590765860", "WesleyChapel", "FL", "33545", "5300", "NoError"]]
response = get_response(f'{HOST}:{PORT}', sample_data, model_name)
try: 
    model_op = response["response_obj"]["outputs"][0]["data"]
    print("Model inference passed for model {model_name}" f" with response {str(model_op)}")
except Exception as e:
    print(f"Model inference failed for model {model_name}"  f"with exception : {e}, \n the server response is {response}")


Making inference call to http://localhost:8000/v2/models/ccf_bmc_model/versions/1/infer with data 
 {'inputs': [{'name': 'IN0', 'shape': [2, 15], 'datatype': 'BYTES', 'data': [['525', '4388509355512107', '2005', '11', '8', '7', '29', '43.42', 'OnlineTransaction', '31551052261259716', 'ONLINE', 'NoState', '0', '4784', 'NoError'], ['1673', '5927292790575653', '2005', '12', '5', '10', '44', '35.75', 'SwipeTransaction', '1913477460590765860', 'WesleyChapel', 'FL', '33545', '5300', 'NoError']]}], 'outputs': [{'name': 'OUT0', 'shape': [2, 1], 'datatype': 'FP64'}]} 

 
Model inference passed for model {model_name} with response [0.0, 1.0]


In [41]:
test_df = pd.DataFrame(sample_data,columns=['User', 'Card', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Amount', 'Use Chip', 'Merchant Name', 'Merchant City',
       'Merchant State', 'Zip', 'MCC', 'Errors?'])
preprocessed_data = preprocess_pipeline.transform(test_df)
model.predict(preprocessed_data)

array([0, 1])

In [42]:
# Shut down the server
!docker rm -f tritonserver

tritonserver


### Conclusion
In this notebook, we walked through the process of detecting credit card fraud using Snap ML Boosting Machine Classifier and Triton Inference Server. We prepared and transformed the synthetic dataset, trained a fraud detection model, and evaluated its performance.

We then deployed the model and preprocessing pipeline using the **IBM Z Accelerated for NVIDIA Triton™ Inference Server** with the **ibmsnapml** backend for efficient, real-time inferencing. This approach ensures seamless deployment, scalability, and consistency in fraud detection predictions, making it suitable for production environments.